In [ ]:
import pandas as np
import pandahouse as ph
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy import stats
import seaborn as sns

присоединяемся к базе данных и отправляем запрос

In [ ]:
connection = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'simulator',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [ ]:
query = '''
SELECT post_id,
       time,
       user_id,
       action
FROM {db}.feed_actions
WHERE (toDate(time) > today()-7) and action = 'view'
'''

In [ ]:
df = ph.read_clickhouse(query, connection=connection)
df

создаем новую табличку только с нужными нам полями

In [ ]:
df_unique_post = df[['post_id','time','action']]

добавляем колонку с временем, когда пост появился первый раз

In [ ]:
df_unique_post = df_unique_post.merge(df_unique_post.groupby('post_id', as_index=False)\
                                      .min().drop('action', axis=1),left_on='post_id', right_on='post_id',how='left')\
                                        .rename(columns={'time_x':'time','time_y':'min_time'})

In [ ]:
df_unique_post.head()

переводим дельтатайм в секунды

In [ ]:
df_unique_post['lifetime'] = (df_unique_post.time - df_unique_post.min_time).apply(lambda x: x.total_seconds())

In [ ]:
df_unique_post.head()

визуализируем

In [ ]:
df_unique_post.hist('lifetime')